In [1]:
import sys
sys.path.append('..')

import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import torch
import time
import pandas as pd
from tqdm.auto import tqdm

# 导入我们自己编写的核心模块
from src.models.model_loader import load_main_llm
from src.utils.data_loader import load_agnews_dataset
from src.pipeline import AhpPipeline
from src.utils.metrics import calculate_accuracy

# 确保PyTorch能使用GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"当前使用的设备: {device}")

当前使用的设备: cuda


In [2]:
# 根据方案设置固定参数
M_CANDIDATES = 10
K_PRUNED = 3
DATASET_NAME = "AG News" # <--- 修改点
NUM_SAMPLES_TO_TEST = 100

# 定义我们要对比的剪枝器列表
PRUNER_TYPES = ['semantic', 'perplexity', 'nli', 'clustering']

In [3]:
# 定义模型加载路径
# local_model_path = "/root/autodl-tmp/vicuna-7b-v1.5" 
local_model_path = "/root/autodl-tmp/circulus_alpaca-7b"
print(f"准备从本地路径加载主模型: {local_model_path}")

# 加载主LLM 
main_model, main_tokenizer = load_main_llm(model_name=local_model_path, use_4bit=True)

# 加载AG News测试集 <--- 修改点
agnews_test_dataset = load_agnews_dataset(split='test')

# 为了方便，我们将数据转换为Pandas DataFrame，并只取前N个样本
agnews_test_df = agnews_test_dataset.to_pandas().head(NUM_SAMPLES_TO_TEST)
print(f"\n成功加载AG News测试集的前 {NUM_SAMPLES_TO_TEST} 条数据。")
agnews_test_df.head()

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


准备从本地路径加载主模型: /root/autodl-tmp/circulus_alpaca-7b
正在加载主模型: /root/autodl-tmp/circulus_alpaca-7b...


The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

主模型加载成功。
正在加载AG News数据集 (test split)...
AG News数据集加载成功。

成功加载AG News测试集的前 100 条数据。


,text,label,label_text
0,Fears for T N pension after talks Unions repre...,2,Business
1,The Race is On: Second Private Team Sets Launc...,3,Sci/Tech
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,Sci/Tech
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,Sci/Tech
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,Sci/Tech


In [4]:
results = []

# 遍历每一种剪枝方法
for pruner_name in PRUNER_TYPES:
    print(f"\n{'='*20} 正在测试剪枝器: {pruner_name.upper()} {'='*20}")
    
    # 初始化AHP流水线，并指定task='ag_news' <--- 关键修改点
    pipeline = AhpPipeline(
        main_model=main_model,
        main_tokenizer=main_tokenizer,
        pruner_name=pruner_name,
        k_val=K_PRUNED,
        m_val=M_CANDIDATES,
        task='ag_news'
    )
    
    predictions = []
    start_time = time.time()
    
    # 遍历AG News测试数据集
    for index, row in tqdm(agnews_test_df.iterrows(), total=len(agnews_test_df), desc=f"Processing with {pruner_name}"):
        sentence = row['text'] # <--- 修改点: AG News的数据列名为'text'
        
        prediction = pipeline.predict_single(sentence)
        predictions.append(prediction)
        
    end_time = time.time()
    
    # 计算指标
    total_time = end_time - start_time
    avg_time_per_sample = total_time / NUM_SAMPLES_TO_TEST
    
    # 使用'label_text'列进行准确率计算
    accuracy = calculate_accuracy(agnews_test_df['label_text'].tolist(), predictions)
    
    # 保存结果
    results.append({
        "筛选规则 (Pruning Rule)": pruner_name,
        "任务准确率 (Task Accuracy)": accuracy,
        "计算开销 (s/sample)": avg_time_per_sample
    })
    
    print(f"剪枝器 '{pruner_name}' 完成测试。")
    print(f" -> 准确率: {accuracy:.4f}")
    print(f" -> 平均耗时: {avg_time_per_sample:.4f} 秒/样本")


==================== 正在测试剪枝器: SEMANTIC ====================
正在加载剪枝模型: sentence-transformers/all-mpnet-base-v2...
剪枝模型加载成功。


Processing with semantic:   0%|          | 0/100 [00:00<?, ?it/s]

剪枝器 'semantic' 完成测试。
 -> 准确率: 0.6800
 -> 平均耗时: 3.9550 秒/样本

==================== 正在测试剪枝器: PERPLEXITY ====================
正在加载主模型: gpt2...
主模型加载成功。


Processing with perplexity:   0%|          | 0/100 [00:00<?, ?it/s]

Calculating PPL:   0%|          | 0/10 [00:00<?, ?it/s]

...

Calculating NLI Scores:   0%|          | 0/10 [00:00<?, ?it/s]

Calculating NLI Scores:   0%|          | 0/10 [00:00<?, ?it/s]

剪枝器 'nli' 完成测试。
 -> 准确率: 0.6900
 -> 平均耗时: 4.0132 秒/样本

==================== 正在测试剪枝器: CLUSTERING ====================
正在加载剪枝模型: sentence-transformers/all-mpnet-base-v2...
剪枝模型加载成功。


Processing with clustering:   0%|          | 0/100 [00:00<?, ?it/s]

剪枝器 'clustering' 完成测试。
 -> 准确率: 0.7000
 -> 平均耗时: 3.9415 秒/样本


In [5]:
# 将结果转换为DataFrame
results_df_agnews = pd.DataFrame(results)

print("\n实验一 (AG News)：最优筛选规则确定 - 结果汇总")
print("=" * 60)
print(f"数据集: {DATASET_NAME}, 测试样本数: {NUM_SAMPLES_TO_TEST}, M={M_CANDIDATES}, K={K_PRUNED}")
print("-" * 60)

# 按照准确率降序排序
results_df_sorted = results_df_agnews.sort_values(by="任务准确率 (Task Accuracy)", ascending=False)
print(results_df_sorted.to_string(index=False))


实验一 (AG News)：最优筛选规则确定 - 结果汇总
数据集: AG News, 测试样本数: 100, M=10, K=3
------------------------------------------------------------
筛选规则 (Pruning Rule)  任务准确率 (Task Accuracy)  计算开销 (s/sample)
         clustering                   0.70         3.941537
         perplexity                   0.69         3.984028
                nli                   0.69         4.013221
           semantic                   0.68         3.954984


In [6]:
# (新增单元格 6) - 保存AG News实验结果

# 确保我们的results目录存在
if not os.path.exists('../results'):
    os.makedirs('../results')
    print("创建 'results' 文件夹。")


# --- 新增逻辑：生成不会被覆盖的文件名 ---
# 1. 定义文件名的基础部分和扩展名
base_path = '../results/experiment_1_agnews_results'
extension = '.csv'
save_path_agnews = f"{base_path}{extension}"
# 2. 检查文件是否存在，如果存在，则在文件名后附加一个数字
counter = 1
while os.path.exists(save_path_agnews):
    # 例如，生成 "experiment_1_agnews_results_1.csv"
    save_path_agnews = f"{base_path}_{counter}{extension}"
    counter += 1
# --- 逻辑结束 ---
# 使用 to_csv 方法保存DataFrame
# index=False 表示我们不把DataFrame的行索引写入到文件中

results_df_sorted.to_csv(save_path_agnews, index=False)

print(f"\n实验结果已成功保存到: {save_path_agnews}")


实验结果已成功保存到: ../results/experiment_1_agnews_results_1.csv
